## Uppgift 1
### Landstatistik

Ni får ett land tilldelat till er grupp. Börja med att anonymisera kolumnen med idrottarnas namn med hashfunktionen SHA-256.

Undersök därefter hur det gått för landet i OS genom tiderna.

Visualisera exempelvis:

- de sporter landet fått flest medaljer i


In [95]:
import pandas as pd
import hashlib as hl
import matplotlib.pyplot as plt
import plotly_express as px


In [96]:
anon_df = pd.read_csv("athlete_events.csv")
anon_df["Name"] = anon_df["Name"].apply(lambda name: hl.sha256(name.encode()).hexdigest())

anon_df.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,3a4eef48434c66b3f14ab0221f6762d0ef7c6135ab2790...,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,a6430cc6630934275dc6283f7e97e9625e6587cdddec7a...,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,9c198b205332c2c8e1542e0f9534b9e270780a41d978ec...,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [97]:
df_gbr = anon_df[anon_df["NOC"] == "GBR"]

df_gbr.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
785,453,068d40aa50057f323cccc9b76161d21e3014635a0bd3b4...,M,24.0,NaN,NaN,Great Britain,GBR,1928 Summer,1928,Summer,Amsterdam,Water Polo,Water Polo Men's Water Polo,NaN
786,453,068d40aa50057f323cccc9b76161d21e3014635a0bd3b4...,M,32.0,NaN,NaN,Great Britain,GBR,1936 Summer,1936,Summer,Berlin,Water Polo,Water Polo Men's Water Polo,NaN
888,509,ad4b525a32fc54f54d6221545f27ff31d81200fa3bc741...,M,17.0,175.0,64.0,Great Britain,GBR,1976 Summer,1976,Summer,Montreal,Swimming,Swimming Men's 100 metres Backstroke,NaN


In [98]:
sport_medals = df_gbr.groupby("Sport", observed=True)[["Medal"]].count() 
top_10_sports = sport_medals.sort_values(by="Medal", ascending=False).head(10)

top_10_sports 

,Medal
Sport,
Athletics,338
Rowing,270
Cycling,182
Hockey,163
Sailing,143
Swimming,127
Shooting,99
Equestrianism,83
Tennis,67


In [99]:
sport_medals = df_gbr.groupby("Sport")["Medal"].count().sort_values(ascending=False)

fig = px.bar(
    top_10_sports,
    x=top_10_sports.index,
    y="Medal",
    labels={"x": "Sport", "Medal": "Number of Medals"},
    title="Number of Medals per Sport",
)

fig.show()

- antal medaljer per OS


In [ ]:
os_medals = df_gbr.groupby("Games", observed=True)[["Medal"]].count()

os_medals.tail(10)

,Medal
Games,
1896 Summer,9
1900 Summer,108
1904 Summer,2
1906 Summer,39
1908 Summer,368
1912 Summer,170
1920 Summer,111
1924 Summer,64
1924 Winter,31


In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html 
# https://stackoverflow.com/questions/72495322/faster-alternative-to-groupby-unstack-then-fillna 
# https://www.w3resource.com/pandas/dataframe/dataframe-unstack.php
os_type_medals = df_gbr.groupby(['Games', 'Medal'], observed=True).size().unstack(fill_value=0) 

os_type_medals.tail(10)

Medal,Bronze,Gold,Silver
Games,,,
1896 Summer,3,3,3
1900 Summer,15,59,34
1904 Summer,0,1,1
1906 Summer,8,9,22
1908 Summer,90,147,131
1912 Summer,59,47,64
1920 Summer,17,52,42
1924 Summer,19,15,30
1924 Winter,11,16,4


In [ ]:
os_type_medals_index = os_type_medals.reset_index()

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.melt.html#pandas.DataFrame.melt
os_type_medals_melted = os_type_medals_index.melt(
    id_vars=["Games"],
    value_vars=["Bronze", "Silver", "Gold"],
    var_name="Type of Medals",
    value_name="Number of Type"
)

# https://plotly.com/python/discrete-color/ # https://i.sstatic.net/lFZum.png
fig = px.bar(
    os_type_medals_melted,
    x="Games",
    y="Number of Type",
    color="Type of Medals",
    title="Number of medals won by GBR on each OS",
    labels={"Games": "Year", "Number of Type": "Number of Medals"},
    color_discrete_map={"Bronze": "tan", "Silver": "silver", "Gold": "gold"}
)

# https://plotly.com/python/bar-charts/
fig.update_layout(
    xaxis_tickangle=90,
    barmode="stack"
)

fig.show()


,Games,Type of Medals,Number of Type
0,1896 Summer,Bronze,3
1,1900 Summer,Bronze,15
2,1904 Summer,Bronze,0
3,1906 Summer,Bronze,8
4,1908 Summer,Bronze,90
...,...,...,...
127,2008 Summer,Gold,31
128,2010 Winter,Gold,1
129,2012 Summer,Gold,48
130,2014 Winter,Gold,1


- histogram över åldrar

In [103]:
print(f"Oldest participant =", df_gbr["Age"].max())
print(f"Youngest participant =", df_gbr["Age"].min())
print(f"Rounding the age difference to divide ages in group aka bins =", round(84/11,0))
print("Each histogram represent groups of 9.125 years, example 11-20, 21-30 and so on")

# https://plotly.com/python/histograms/
fig = px.histogram(
    df_gbr,
    x="Age",
    nbins=8,
    title="OS-GBR: Participants Ages",
    labels={"Age": "Age Bracket", "Count" : "Number of Individuals"}
)

fig.update_layout(
    yaxis_title="Number of Individuals",
)

fig.show()

Oldest participant = 84.0
Youngest participant = 11.0
Rounding the age difference to divide ages in group aka bins = 8.0
Each histogram represent groups of 9.125 years, example 11-20, 21-30 and so on


## Uppgift 2 
### Sportstatistik

Välj 2-4 sporter och skapa lämpliga grafer/diagram för att visualisera exempelvis:
- medaljfördelning mellan länder i sporterna

In [ ]:
medals = anon_df[anon_df["Medal"].notna()].reset_index(drop=True)

# https://plotly.com/python/bar-charts/
def sport_medals(sports):
    
    sport = medals[medals["Sport"] == sports].reset_index(drop=True)
    sport = sport.groupby(["NOC", "Medal"]).size().reset_index(name="Medal Count")

    fig = px.bar(sport, x="NOC", y="Medal Count", color="Medal", color_discrete_map = {"Bronze": "tan", "Silver": "silver", "Gold": "gold"}, title="Countries with number of medals for " + sports, labels={"NOC": "Country", "Medal Count": "Number of Medals"})

    return fig.show()

sport_medals_football = sport_medals("Football")
sport_medals_swimming = sport_medals("Swimming")
sport_medals_boxing = sport_medals("Boxing")
sport_medals_rugby = sport_medals("Rugby")

- åldersfördelning i sporterna

In [ ]:
def sport_age_people(sports):
    sport = anon_df[anon_df["Sport"] == sports].reset_index(drop=True)
    sport = sport.groupby(["Age"]).size().reset_index(name="Age Count")
    fig = px.bar(sport, x="Age", y="Age Count", color="Age", title="Number of individiuals with specific age for " + sports, labels={"Age Count": "Number of athletes"})
    
    return fig.show()
  
sport_age_football = sport_age_people("Football")
sport_age_swimming = sport_age_people("Swimming")
sport_age_boxing = sport_age_people("Boxing")
sport_age_rugby = sport_age_people("Rugby")

## Källa
### Många uppgifter löstes med inspiration av koden från repo nedan
https://github.com/pr0fez/AI24-Databehandling 
